# HW6 - Logistic Regression & SVM
##### 負責助教：張逸群 toosyou.tw@gmail.com ~~對又是我~~

## Data - Breast cancer wisconsin (diagnostic) dataset
* Number of Instances: 569
* Number of Attributes: 30 numeric, predictive attributes and the class
* Attribute Information:
    * radius (mean of distances from center to points on the perimeter)
    * texture (standard deviation of gray-scale values)
    * perimeter
    * area
    * smoothness (local variation in radius lengths)
    * compactness (perimeter^2 / area - 1.0)
    * concavity (severity of concave portions of the contour)
    * concave points (number of concave portions of the contour)
    * symmetry
    * fractal dimension (“coastline approximation” - 1)
    * The mean, standard error, and “worst” or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.
    * **class**:
        * WDBC-Malignant
        * WDBC-Benign

## 評分標準
### 原始分數
* 上課繳交：20%
    * 在當天 16:20 前上傳至 new E3 作業，請將以下打包成 .zip 上傳：
        * 程式原始檔 (.ipynb）
        * 程式 .html 檔
    * 並填寫表單上傳程式執行結果
        * 請上傳 .html 檔案至表單
        * __請統一檔名 <學號>-<姓名>.html__
        * [表單連結](https://docs.google.com/forms/d/e/1FAIpQLSdlbNU_u9kb1QpCdXsgX42gx6UcRYCtVnX4E0xVlkjYyEOgug/viewform)
* 程式分數：80%

### 調整分數
* 你的調整分數 = (你的原始分數 - 全部人原始分數平均) x (15. / 全部人原始分數標準差) + 80.

姓名: 林哲宇
學號: 0616018

## Packages that Might Be Used

In [81]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# ignore FutureWarning, 'cause it's so annoying 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
%matplotlib inline

## Load Data - 20%

#### 請使用 `pd.read_csv` 將 `data.csv` 讀入成 `DataFrame` 至 `df` - 10%

In [82]:
df = pd.read_csv('data.csv')

In [83]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


#### 請將除了 `id` 和 `diagnosis` 外的 column 指派為 `X`（此為我們使用的 feature），並將 `diagnosis` column 指派為 `y`（此為我們將預測的目標）- 10%

In [84]:
X = df.drop(columns=['id', 'diagnosis', 'Unnamed: 32'])
y = pd.DataFrame(df, columns=['diagnosis'])

In [85]:
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [86]:
y.head()

,diagnosis
0,M
1,M
2,M
3,M
4,M


## Train-Test Split - 10%

#### 請使用 `train_test_split` 將 `X` 與 `y` 分成 Training Set 以及 Testing Set - 10%
* test_size = 0.33
* random_state = 42

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

## Logistic Regression - 30%

#### 請使用 [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 及 **`Training Set`** 對 LogisticRegression Model 進行 [Hyperparamter Search](https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator) - 10%
* Search 範圍
    * penalty： l1, l2
    * C: 1, 10

In [88]:
parameters = {'C':[1, 10], 'penalty':['l1', 'l2']}
lr_grid = GridSearchCV(LogisticRegression(), parameters, return_train_score=True)
lr_grid = lr_grid.fit(X_train, y_train)

/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zeze/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, 

![meme1](https://imgur.com/QzAOB3V.jpg)

#### 將 Search 結果印出 - 0%

In [89]:
pd.DataFrame(lr_grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
2,0.226612,0.057483,0.001343,0.000075,10,l1,"{'C': 10, 'penalty': 'l1'}",0.976562,0.992126,0.936508,0.968504,0.023375,1,0.988142,0.972441,0.988235,0.982940,0.007424
3,0.006700,0.000777,0.001484,0.000318,10,l2,"{'C': 10, 'penalty': 'l2'}",0.976562,0.976378,0.928571,0.960630,0.022535,2,0.968379,0.960630,0.984314,0.971108,0.009859
0,0.146479,0.024560,0.001295,0.000132,1,l1,"{'C': 1, 'penalty': 'l1'}",0.937500,0.984252,0.920635,0.947507,0.026880,3,0.964427,0.948819,0.968627,0.960624,0.008522
1,0.006852,0.001170,0.001656,0.000475,1,l2,"{'C': 1, 'penalty': 'l2'}",0.937500,0.976378,0.920635,0.944882,0.023311,4,0.952569,0.948819,0.956863,0.952750,0.003286


#### 請回答以下問題 - 10%

1. 請問 Search 結果中，表現最好的是哪一組參數？ - 5%
    * {'C': 10, 'penalty': 'l1'} 那組
2. 承上，請問是以何作為依據？ - 5%
    * mean_test_score

#### 自 `lr_grid` 中取出最好的模型，並且使用完整的 **Training Set** 重新訓練，最後使用 **Testing Set** 計算 Accuracy - 10%

In [90]:
from sklearn.metrics import accuracy_score

best_lr_clf = LogisticRegression(C=10, penalty='l1')
best_lr_clf.fit(X_train, y_train)
y_pred = best_lr_clf.predict(X_test)

print(best_lr_clf)
print('lr acc:', accuracy_score(y_test, y_pred))

/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
lr acc: 0.9627659574468085


## SVM - 30%

#### 請使用 [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 及 **`Training Set`** 對 [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) 進行 [Hyperparamter Search](https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator) - 10%
* Search 範圍
    * kernel： linear, rbf
    * C: 1, 10

In [91]:
parameters = {'kernel':['linear', 'rbf'], 'C':[1, 10]}
svc_grid = GridSearchCV(SVC(), parameters, return_train_score=True)
svc_grid = svc_grid.fit(X_train, y_train)

/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

#### 將 Search 結果印出 - 0%

In [92]:
pd.DataFrame(svc_grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
2,2.473686,1.201664,0.001586,0.000024,10,linear,"{'C': 10, 'kernel': 'linear'}",0.968750,0.984252,0.944444,0.965879,0.016347,1,0.968379,0.960630,0.996078,0.975029,0.015217
0,0.653448,0.284936,0.001870,0.000436,1,linear,"{'C': 1, 'kernel': 'linear'}",0.953125,0.960630,0.928571,0.947507,0.013660,2,0.968379,0.956693,0.980392,0.968488,0.009675
1,0.009390,0.001079,0.003769,0.000609,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.617188,0.622047,0.619048,0.619423,0.002005,3,1.000000,1.000000,1.000000,1.000000,0.000000
3,0.008244,0.000070,0.003018,0.000075,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.617188,0.622047,0.619048,0.619423,0.002005,3,1.000000,1.000000,1.000000,1.000000,0.000000


#### 請回答以下問題 - 10%

1. 請問 Search 結果中，表現最好的是哪一組參數？ - 5%
    * {'C': 10, 'kernel': 'linear'} 那組
2. 承上，請問是以何作為依據？ - 5%
    * mean_test_score

![出作業](https://imgur.com/qmMHznz.jpg)

#### 自 `svc_grid` 中取出最好的模型，並且使用完整的 **Training Set** 重新訓練，最後使用 **Testing Set** 計算 Accuracy - 10%

In [93]:
best_svc_clf = SVC(C = 10, kernel = 'linear')
best_svc_clf = best_svc_clf.fit(X_train, y_train)
y_pred2 = best_svc_clf.predict(X_test)

print(best_svc_clf)
print('svc acc:', accuracy_score(y_test, y_pred2))

/home/zeze/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
svc acc: 0.9414893617021277


## Prediction - 10%

#### 假設現在有一個新的病人資料如下，請使用上面建立的 `best_lr_clf` 以及 `best_svc_clf` 分別對他進行預測，並印出預測結果 - 10%

In [94]:
X_new = [[11.61,
 16.02,
 75.46,
 408.2,
 0.1088,
 0.1168,
 0.07097,
 0.044969999999999996,
 0.1886,
 0.0632,
 0.2456,
 0.7339,
 1.6669999999999998,
 15.89,
 0.005884,
 0.02005,
 0.026310000000000004,
 0.013040000000000001,
 0.01848,
 0.001982,
 12.64,
 19.67,
 81.93,
 475.7,
 0.1415,
 0.217,
 0.2302,
 0.1105,
 0.2787,
 0.07427]]

In [95]:
print('lr prediction:', best_lr_clf.predict(X_new))
print('svc prediction:', best_svc_clf.predict(X_new))

lr prediction: ['B']
svc prediction: ['B']


![ml_model](https://imgur.com/JsbckWU.jpg)

# 歡迎跟 ~~平常沒有人可以說話的~~ 助教縮縮話 <3

* 